In [ ]:
import sys; sys.path.extend(['..', '../validations/'])
import MeshFEM
import mesh, elastic_sheet
from tri_mesh_viewer import TriMeshViewer, PointCloudViewer
import py_newton_optimizer, fd_validation, benchmark
from matplotlib import pyplot as plt
import meshing, time
import numpy as np
from io_redirection import suppress_stdout as so

In [ ]:
import sim_utils, sheet_convergence

In [ ]:
es = sheet_convergence.getSheet(0.05, maxArea=2e-2, L=12, useNeoHookean=True)

In [ ]:
es.setDeformedPositions(sheet_convergence.Phi(es.getRestPositions()))
es.initializeMidedgeNormals()

In [ ]:
leftThetas   = sim_utils.getBBoxVars(es, sim_utils.BBoxFace.MIN_X, displacementComponents=[])
rightThetas  = sim_utils.getBBoxVars(es, sim_utils.BBoxFace.MAX_X, displacementComponents=[])
leftEdgeVars = sim_utils.getBBoxVars(es, sim_utils.BBoxFace.MIN_X)
rightEdgeVars= sim_utils.getBBoxVars(es, sim_utils.BBoxFace.MAX_X)

In [ ]:
thetas = es.getThetas()

In [ ]:
v_plate = TriMeshViewer(es, wireframe=True, width=800, height=600)
v_plate.show()

In [ ]:
v_plate.setDarkMode(True)

In [ ]:
v_plate.resetCamera()

In [ ]:
plate_normalVis = None
def updateNormalView():
    return
    global plate_normalVis
    v_plate.subViews = []
    plate_normalVis = PointCloudViewer(es.edgeMidpoints(), vectorField=es.midedgeNormals(), superView=v_plate)
    plate_normalVis.arrowSize = 20
updateNormalView()

In [ ]:
es.setThetas(thetas)
updateNormalView()

In [ ]:
accumTurn = 0

In [ ]:
updatedThetas = es.getThetas()
delta = np.pi / 48
updatedThetas[np.array(leftThetas) - es.thetaOffset()] -= delta
updatedThetas[np.array(rightThetas) - es.thetaOffset()] -= delta
accumTurn += delta
es.setThetas(updatedThetas)
#updateNormalView()

def iter_cb(prob, it):
    v_plate.update()
    #updateNormalView()
opts = py_newton_optimizer.NewtonOptimizerOptions()
opts.niter = 1000
opts.gradTol = 1e-10
es.computeEquilibrium([], leftEdgeVars + rightEdgeVars, opts=opts, cb=iter_cb)
#v_plate.update()

In [ ]:
es.energy(), es.energy(etype=es.EnergyType.Membrane), es.energy(etype=es.EnergyType.Bending)